In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline
# %seaborn inline

#  Import and read the charity_data.csv.
import pandas as pd 
fire_df = pd.read_csv("Data/California_Fire_Incidents.csv")
fire_df[fire_df["CountyIds"] == '6,11,17,23']["Name"]

1054    Ranch Fire (Mendocino Complex)
1055    Ranch Fire (Mendocino Complex)
1056    Ranch Fire (Mendocino Complex)
1057    Ranch Fire (Mendocino Complex)
Name: Name, dtype: object

In [2]:
fire_df = fire_df.drop(["Active", 
"AdminUnit", 
"AirTankers",
"CanonicalUrl", 
"ConditionStatement", 
"ControlStatement",
"CrewsInvolved",
"Dozers",
"Engines",
"Helicopters"
"Extinguished",
"Fatalities",
"Featured",
"Final",
"FuelType",
"Injuries",
"Location",
# "Name",
"PercentContained",
"Public",
"SearchDescription",
"SearchKeywords",
"Status",
"StructuresDamaged",
"StructuresDestroyed",
"StructuresEvacuated",
"StructuresThreatened",
"UniqueId",
"Updated"
], axis=1)
fire_df.head()

KeyError: "['HelicoptersExtinguished'] not found in axis"

In [ ]:
# LOOK AT UNDECIDED COUNTIES
[(column, fire_df[column].nunique()) for column in fire_df]

In [ ]:
[(column, fire_df[column].unique()) for column in fire_df[["Counties","CountyIds"]]]

In [ ]:
fire_df[fire_df["CountyIds"] == '6,11,17,23']

In [ ]:
fire_df[fire_df["CountyIds"] == '6']

In [ ]:
env_df = pd.read_csv("Data/all_env_conditions.csv")

In [ ]:
env_df = env_df.drop(["Target", "CIMIS Region"], axis=1)
env_df.head()

In [ ]:
electric_df = pd.read_csv("Data/ElectricityByCounty CALIFORNIA Usage in Millions of kWh.csv")

In [ ]:
# Find the totals for each sector, and then drop that column.
electric_df = electric_df[electric_df["Sector"] == "Total"].drop(["Sector"], axis=1)

# Drop the years before 2013
electric_df = electric_df.iloc[:, :8]

In [ ]:
electric_df.head()

In [ ]:
housing_df = pd.read_csv("Data/housing.csv")

In [ ]:
housing_df = housing_df.drop(["housing_median_age", "total_rooms","total_bedrooms"], axis=1)
housing_df.head()

In [ ]:
def boxplot_it(df, column_number):
    print("THERE ARE {} NUMERICAL COLUMNS IN THIS DF AVAILABLE".format( len(df.select_dtypes(include='number').columns)) )
    number_df = pd.DataFrame(df.select_dtypes(include='number').iloc[:,column_number])
    [plt.boxplot(x=number_df[column].dropna(), labels=[column]) for column in number_df]

In [ ]:
boxplot_it(fire_df,4 )
